In [1]:
import datetime
import numpy as np
import math
import os
import pandas as pd
import re
import seaborn as sns
import json
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    

os.chdir(r'C:\Users\User\GitHub\WebET_Analysis')
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [2]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Raw Data

## Search for specific subjects

In [3]:
#path = 'data_cognition'
#subject_files = os.listdir(path)
#for i in range(0, len(subject_files)):
#    thisSubject = open(path + "/" + subject_files[i]).read()
#    if thisSubject.find('5c0e4e3d87e876000151cfec') > (-1):
#        print(subject_files[i])

## Read CSV from String

In [4]:
def cleanhtml(raw_html):
    # https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
    cleanr = re.compile('<.*?>')
    cleanText = re.sub(cleanr, '', raw_html)
    return cleanText

def cleanETText(text):
    textWithinBrackets = re.findall(re.compile('\[.*?\]'), text)
    output = text
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = re.sub(",", "$", old)
        output = output.replace(old, new)
    return output

def cleanSurveyText(text):
    output = text
    textWithinBrackets = re.findall(re.compile('\{.*?\}'), text)
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = old.replace(',', '§')
        output = output.replace(old, new)
    return output

def compileData(path):
    subject_files = os.listdir(path)
    all_subjects = []
    for i in range(0, len(subject_files)):
        csv_thisSubject = open(path + "/" + subject_files[i]).read()
        csv_thisSubject = cleanhtml(csv_thisSubject)
        csv_thisSubject = cleanETText(csv_thisSubject)
        csv_thisSubject = cleanSurveyText(csv_thisSubject)
        all_subjects.append(pd.read_csv(StringIO(csv_thisSubject)))
    output = pd.concat(all_subjects).reset_index(drop=True)
    return output

data_raw = compileData("data_prolific")

# Custom modifications
data_raw['run_id'].unique()

array([1, 103, 105, 106, 108, 11, 117, 12, 121, 124, 125, 126, 128, 13,
       130, 131, 14, 19, 24, 25, 28, 3, 30, 32, 36, 37, 38, 4, 41, 42, 43,
       45, 47, 48, 49, 5, 50, 51, 54, 56, 57, 58, 59, 6, 61, 63, 66, 67,
       7, 70, 72, 74, 75, 80, 81, 83, 85, 88, 9, 90, 91, 92, 93, 94, 96,
       97, 98, 99], dtype=object)

In [5]:
data_yang = compileData("data_yang2020WG")
# Custom modifications
data_yang.loc[0:515, 'run_id'] = 0
data_yang = data_yang.loc[~data_yang['run_id'].isin([4, 18]), :]
data_yang = data_yang.rename(columns={'eyeData': 'et_data'})
data_yang['run_id'] = data_yang['run_id'] + 1000
print(data_yang['run_id'].unique())
data_raw = data_raw.append(data_yang.reset_index(drop=True))

[1000 1001 1011 1013 1014 1015 1016 1017 1019 1002 1020 1021 1003 1007
 1008 1009 1022 1024]


In [6]:
data_milieu = compileData("data_milieu")
data_milieu['run_id'] = data_milieu['run_id'] + 2000
print(data_milieu['run_id'].unique())
data_raw = data_raw.append(data_milieu.reset_index(drop=True))

[2011 2012 2013 2002 2008 2009]


### Exclude empty studies

In [7]:
data_raw = data_raw.loc[data_raw['trial_index']>0, :]

### survey data

In [8]:
def cleanOptionalNote(text):
    optionalNoteText = re.findall(re.compile('optionalNote":.*?\}'), text)
    if len(optionalNoteText) < 1:
        output = text
    else:
        old = optionalNoteText[0]
        new = old.replace('§', ' ')
        output = text.replace(old, new)
    return output


def surveyStringToFrame(subject, string):
    string = cleanOptionalNote(string)
    string = re.sub("""{""", '', string)
    string = re.sub("""}""", '', string)
    string = re.sub('"', '', string)
    string = re.sub('§', '$', string)
    output = pd.read_csv(StringIO(string),
                         sep=":",
                         lineterminator="$",
                         header=None,
                         index_col=0
                         ).transpose()
    return output


def surveyData_thisSubject(data):
    df_thisSubject = data.loc[
        (pd.notna(data["responses"])) &
        (data["responses"] != '"'), :].reset_index()
    subject = df_thisSubject['run_id'].unique()[0]
    output = []
    for i in range(0, len(df_thisSubject)):
        output.append(
            surveyStringToFrame(subject,
                                df_thisSubject.loc[i, 'responses'])
        )

    output = pd.concat(output, axis=1)
    output['run_id'] = subject
    return output

def addSurveyData(data):
    surveyData_allSubjects = pd.DataFrame(columns=[
            'prolificID', 'age', 'gender', 'ethnic', 'sight', 
            'glasses', 'degree', 'eyeshadow', 'masquara', 'eyeliner', 
            'browliner', 'vertPosition', 'triedChin', 'keptHead', 
            'optionalNote', 'run_id'])

    for subject in data['run_id'].unique():
        surveyData_allSubjects = \
            surveyData_allSubjects.append(
                surveyData_thisSubject(
                    data.loc[data['run_id']==subject, ['run_id', 'responses']]
                )
            )
    # print(surveyData_allSubjects)
    
    data = data.merge(surveyData_allSubjects, on='run_id', how='left')
    data = data.drop(columns='responses')
    return(data)
    
data_raw = addSurveyData(data_raw)
print(data_raw['run_id'].unique())
print(data_raw.columns)

[1 103 106 108 11 12 124 125 126 128 13 130 131 14 19 24 25 28 30 32 36 37
 38 4 41 42 43 45 47 48 49 5 54 56 58 59 6 61 63 66 67 7 70 72 74 75 80 81
 83 85 88 9 90 91 92 93 94 96 97 98 99 1000 1001 1011 1014 1015 1016 1017
 1019 1002 1020 1021 1003 1007 1008 1009 1022 1024 2011 2012 2013 2002
 2008 2009]
Index(['run_id', 'condition', 'rt', 'stimulus', 'key_press', 'trial_type',
       'trial_index', 'time_elapsed', 'internal_node_id', 'subject',
       'chinFirst', 'choiceTask_amountLeftFirst', 'webcam_label', 'webcam_fps',
       'webcam_height', 'webcam_width', 'button_pressed', 'window_width',
       'window_height', 'chin', 'success', 'x_pos', 'y_pos', 'task_nr',
       'et_data', 'trial_duration', 'option_topLeft', 'option_bottomLeft',
       'option_topRight', 'option_bottomRight', 'recorded_at', 'ip',
       'user_agent', 'device', 'browser', 'browser_version', 'platform',
       'platform_version', 'Unnamed: 2', 'chosenAmount', 'chosenDelay',
       'webcam_Id', 'webcam_aspect

In [9]:
def convertToNumeric(data, columns):
    data[columns] = data[columns].apply(pd.to_numeric, errors='coerce')
    return data


data_raw = convertToNumeric(data_raw, ['age'])
data_raw['degree'] = data_raw['degree'].replace(
        [
            'College / Undergraduate / Bachelor',
            'High School',
            'Graduate / PhD / Master',
            'Middle School'
        ],
    [
            'college',
            'highSchool',
            'grad',
            'middle'
        ]
)

columns = [
            'chinFirst',
            'eyeshadow', 
            'masquara',
            'eyeliner',
            'browliner',
            'triedChin', 
            'keptHead',
        ]

data_raw[columns] = data_raw[columns].replace({'no': 0, 'yes': 1}) 

data_raw['run_id'].unique()

array([1, 103, 106, 108, 11, 12, 124, 125, 126, 128, 13, 130, 131, 14, 19,
       24, 25, 28, 30, 32, 36, 37, 38, 4, 41, 42, 43, 45, 47, 48, 49, 5,
       54, 56, 58, 59, 6, 61, 63, 66, 67, 7, 70, 72, 74, 75, 80, 81, 83,
       85, 88, 9, 90, 91, 92, 93, 94, 96, 97, 98, 99, 1000, 1001, 1011,
       1014, 1015, 1016, 1017, 1019, 1002, 1020, 1021, 1003, 1007, 1008,
       1009, 1022, 1024, 2011, 2012, 2013, 2002, 2008, 2009], dtype=object)

# data_trial

In [10]:
data_trial = data_raw.loc[
    :,
    [
        'run_id', 'subject', 'prolificID', 'chinFirst', 
        'trial_index',
        'trial_type', 'task_nr', 
        'rt', 'stimulus', 'key_press', 
        'time_elapsed', 'trial_duration', 'recorded_at',
        'window_width', 'window_height', 'success', 

        'chin', 'x_pos', 'y_pos', 
        
        'choiceTask_amountLeftFirst', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight', 
        'chosenAmount', 'chosenDelay',
    ]
]

## Clean

### Failed Webgazer Setup

In [11]:
subjects_failedSetups = []
for subject in data_trial['run_id'].unique():
    maxTrialIndex = data_trial.loc[data_raw['run_id']==subject, ['trial_index']].max()
    if int(maxTrialIndex) < 15:
        subjects_failedSetups.append(subject)

data_trial.loc[data_trial['run_id'].isin(subjects_failedSetups), 
    [
        'run_id', 'prolificID'
    ]
].drop_duplicates()    

,run_id,prolificID
1036,106,5d485e8415055400194b707f
1038,108,55b237e6fdf99b19ea79d2f7
2079,124,56b7a271e77ebe000bbeff49
4684,14,5ec2cdfd1a17930ddf6f1443
12984,49,5ecfc227f036c902457fc44c
13507,54,5f94d146f65f56336a61cc16
18338,72,600324757b530e39903effc0
20414,83,600324757b530e39903effc0
20935,88,5ecfc227f036c902457fc44c
25608,99,55b237e6fdf99b19ea79d2f


In [12]:
data_trial = data_trial.loc[~data_trial['run_id'].isin(subjects_failedSetups), :]
print(data_trial['run_id'].unique())

[1 103 11 12 125 126 128 13 130 131 19 24 25 28 30 32 36 37 38 4 41 42 43
 45 47 48 5 56 58 59 6 61 63 66 67 7 70 74 75 80 81 85 9 90 91 92 93 94 96
 97 98 1000 1011 1021 1003 1008 2011 2012 2013 2002 2008 2009]


### Duplicate Prolific ID's

In [13]:
data_trial.loc[data_trial['run_id']==2002, 'prolificID'] = 'Tim2'

duplicates = data_trial.loc[:, ['prolificID', 'trial_index']].duplicated()
duplicateSubjects = data_trial.loc[duplicates, ['run_id', 'prolificID', 'trial_index']]

if len(duplicateSubjects) > 0:
    print('! Attention: Duplicate subjects: Check out the following: \n')
    print(duplicateSubjects)
else:
    print('Success: No duplicate subjects found')

Success: No duplicate subjects found


### Choice data

In [14]:
# Cleaning
data_trial.loc[: , 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ] = data_trial.loc[:, 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ].replace(['Today', 'Tomorrow', '7 days', '15 days', '30 days', '90 days', '180 days'], 
             [0, 1, 7, 15, 30, 90, 180]) \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) 

### Numerics

In [15]:
data_trial = convertToNumeric(data_trial, 
    [
        'run_id', 'subject', 'chinFirst', 'chin', 'task_nr', 'trial_index', # Int
        'key_press', 
        'x_pos', 'y_pos', 'time_elapsed', 'trial_duration',
        'rt',
        'window_width', 'window_height', 
        'option_topLeft',
        'option_bottomLeft', 
        'option_topRight', 
        'option_bottomRight', 
        'chosenAmount', 
        'chosenDelay'
    ]
)

# data_et

## Compile

In [16]:
def reformatYang(text):
    text = text.replace('$', ',')
    text = text.replace("relative-x", "x")
    text = text.replace("relative-y", 'y')
    text = text.replace('elapse-time', 't')
    text = (text[11:len(text)-1])
    return(text)

for subject in data_yang['run_id'].unique():
    for i in data_raw.loc[
        (data_raw['run_id']==subject) & 
        (pd.notna(data_raw['et_data'])) &
        ~(data_raw['et_data'].isin(['"', 'nan'])), 
        :].index:
        print('Reformat index: ' + str(i))
        data_raw.loc[i, 'et_data'] = reformatYang(data_raw.loc[i, 'et_data'])

Reformat index: 25626
Reformat index: 25628
Reformat index: 25630
Reformat index: 25632
Reformat index: 25634
Reformat index: 25636
Reformat index: 25638
Reformat index: 25640
Reformat index: 25642
Reformat index: 25644
Reformat index: 25646
Reformat index: 25648
Reformat index: 25650
Reformat index: 25652
Reformat index: 25654
Reformat index: 25656
Reformat index: 25658
Reformat index: 25660
Reformat index: 25662
Reformat index: 25664
Reformat index: 25666
Reformat index: 25668
Reformat index: 25670
Reformat index: 25672
Reformat index: 25674
Reformat index: 25676
Reformat index: 25678
Reformat index: 25680
Reformat index: 25682
Reformat index: 25684
Reformat index: 25686
Reformat index: 25688
Reformat index: 25690
Reformat index: 25692
Reformat index: 25694
Reformat index: 25696
Reformat index: 25698
Reformat index: 25700
Reformat index: 25702
Reformat index: 25713
Reformat index: 25714
Reformat index: 25716
Reformat index: 25717
Reformat index: 25719
Reformat index: 25720
Reformat i

Reformat index: 26420
Reformat index: 26421
Reformat index: 26423
Reformat index: 26424
Reformat index: 26426
Reformat index: 26427
Reformat index: 26429
Reformat index: 26430
Reformat index: 26432
Reformat index: 26433
Reformat index: 26435
Reformat index: 26436
Reformat index: 26438
Reformat index: 26439
Reformat index: 26441
Reformat index: 26442
Reformat index: 26444
Reformat index: 26445
Reformat index: 26447
Reformat index: 26448
Reformat index: 26450
Reformat index: 26451
Reformat index: 26453
Reformat index: 26454
Reformat index: 26456
Reformat index: 26457
Reformat index: 26459
Reformat index: 26460
Reformat index: 26462
Reformat index: 26463
Reformat index: 26465
Reformat index: 26466
Reformat index: 26468
Reformat index: 26469
Reformat index: 26471
Reformat index: 26472
Reformat index: 26474
Reformat index: 26475
Reformat index: 26477
Reformat index: 26478
Reformat index: 26480
Reformat index: 26481
Reformat index: 26483
Reformat index: 26484
Reformat index: 26486
Reformat i

Reformat index: 27177
Reformat index: 27179
Reformat index: 27180
Reformat index: 27182
Reformat index: 27183
Reformat index: 27185
Reformat index: 27186
Reformat index: 27188
Reformat index: 27189
Reformat index: 27191
Reformat index: 27192
Reformat index: 27194
Reformat index: 27195
Reformat index: 27197
Reformat index: 27198
Reformat index: 27200
Reformat index: 27201
Reformat index: 27203
Reformat index: 27204
Reformat index: 27206
Reformat index: 27207
Reformat index: 27209
Reformat index: 27210
Reformat index: 27237
Reformat index: 27239
Reformat index: 27241
Reformat index: 27243
Reformat index: 27245
Reformat index: 27247
Reformat index: 27249
Reformat index: 27251
Reformat index: 27253
Reformat index: 27255
Reformat index: 27257
Reformat index: 27259
Reformat index: 27261
Reformat index: 27263
Reformat index: 27265
Reformat index: 27267
Reformat index: 27269
Reformat index: 27271
Reformat index: 27273
Reformat index: 27275
Reformat index: 27277
Reformat index: 27279
Reformat i

Reformat index: 27986
Reformat index: 27987
Reformat index: 27989
Reformat index: 27990
Reformat index: 27992
Reformat index: 27993
Reformat index: 27995
Reformat index: 27996
Reformat index: 27998
Reformat index: 27999
Reformat index: 28001
Reformat index: 28002
Reformat index: 28004
Reformat index: 28005
Reformat index: 28007
Reformat index: 28008
Reformat index: 28011
Reformat index: 28012
Reformat index: 28014
Reformat index: 28015
Reformat index: 28017
Reformat index: 28018
Reformat index: 28020
Reformat index: 28021
Reformat index: 28023
Reformat index: 28024
Reformat index: 28026
Reformat index: 28027
Reformat index: 28029
Reformat index: 28030
Reformat index: 28032
Reformat index: 28033
Reformat index: 28035
Reformat index: 28036
Reformat index: 28038
Reformat index: 28039
Reformat index: 28041
Reformat index: 28042
Reformat index: 28044
Reformat index: 28045
Reformat index: 28047
Reformat index: 28048
Reformat index: 28050
Reformat index: 28051
Reformat index: 28053
Reformat i

In [17]:
def textToDataframe(text):
    text = text.replace('$', ',')
    dataframe = pd.read_json(text, orient='records')
    return(dataframe)

def extractEyetrackingData(data):
    data_eyetracking = pd.DataFrame(columns=['x', 'y', 't'])
    data["et_data"] = data['et_data'].apply(str)

    for i in data.loc[
        (pd.notna(data['et_data'])) & 
        ~(data['et_data'].isin(['"', '[]', 'nan'])), 
        :].index:
        
        print('Start decoding Index ' + str(i))
        df = textToDataframe(data.loc[i, 'et_data'])        

        df["t_task"] = (df.loc[:, "t"] - df.loc[0, "t"])
        df['run_id'] = data.loc[i, 'run_id']
        df['trial_index'] = data.loc[i, 'trial_index']

        data_eyetracking = data_eyetracking.append(pd.DataFrame(data=df), ignore_index=True)
        print('Index ' + str(i) + ' extracted')
    return(data_eyetracking)

data_et = extractEyetrackingData(data_raw.loc[data_raw['run_id'].isin(data_trial['run_id'].unique()), :])

Start decoding Index 17


<ipython-input-17-80ee6abdb188>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["et_data"] = data['et_data'].apply(str)


Index 17 extracted
Start decoding Index 19
Index 19 extracted
Start decoding Index 21
Index 21 extracted
Start decoding Index 23
Index 23 extracted
Start decoding Index 25
Index 25 extracted
Start decoding Index 27
Index 27 extracted
Start decoding Index 29
Index 29 extracted
Start decoding Index 31
Index 31 extracted
Start decoding Index 33
Index 33 extracted
Start decoding Index 35
Index 35 extracted
Start decoding Index 37
Index 37 extracted
Start decoding Index 39
Index 39 extracted
Start decoding Index 41
Index 41 extracted
Start decoding Index 43
Index 43 extracted
Start decoding Index 45
Index 45 extracted
Start decoding Index 47
Index 47 extracted
Start decoding Index 49
Index 49 extracted
Start decoding Index 51
Index 51 extracted
Start decoding Index 53
Index 53 extracted
Start decoding Index 55
Index 55 extracted
Start decoding Index 57
Index 57 extracted
Start decoding Index 59
Index 59 extracted
Start decoding Index 61
Index 61 extracted
Start decoding Index 63
Index 63 ex

Index 360 extracted
Start decoding Index 361
Index 361 extracted
Start decoding Index 363
Index 363 extracted
Start decoding Index 364
Index 364 extracted
Start decoding Index 366
Index 366 extracted
Start decoding Index 367
Index 367 extracted
Start decoding Index 369
Index 369 extracted
Start decoding Index 370
Index 370 extracted
Start decoding Index 372
Index 372 extracted
Start decoding Index 373
Index 373 extracted
Start decoding Index 375
Index 375 extracted
Start decoding Index 376
Index 376 extracted
Start decoding Index 378
Index 378 extracted
Start decoding Index 379
Index 379 extracted
Start decoding Index 381
Index 381 extracted
Start decoding Index 382
Index 382 extracted
Start decoding Index 393
Index 393 extracted
Start decoding Index 395
Index 395 extracted
Start decoding Index 397
Index 397 extracted
Start decoding Index 399
Index 399 extracted
Start decoding Index 401
Index 401 extracted
Start decoding Index 403
Index 403 extracted
Start decoding Index 405
Index 405 

Index 792 extracted
Start decoding Index 793
Index 793 extracted
Start decoding Index 795
Index 795 extracted
Start decoding Index 796
Index 796 extracted
Start decoding Index 798
Index 798 extracted
Start decoding Index 799
Index 799 extracted
Start decoding Index 801
Index 801 extracted
Start decoding Index 802
Index 802 extracted
Start decoding Index 804
Index 804 extracted
Start decoding Index 805
Index 805 extracted
Start decoding Index 807
Index 807 extracted
Start decoding Index 808
Index 808 extracted
Start decoding Index 810
Index 810 extracted
Start decoding Index 811
Index 811 extracted
Start decoding Index 813
Index 813 extracted
Start decoding Index 814
Index 814 extracted
Start decoding Index 816
Index 816 extracted
Start decoding Index 817
Index 817 extracted
Start decoding Index 819
Index 819 extracted
Start decoding Index 820
Index 820 extracted
Start decoding Index 822
Index 822 extracted
Start decoding Index 823
Index 823 extracted
Start decoding Index 825
Index 825 

...

Index 28801 extracted
Start decoding Index 28803
Index 28803 extracted
Start decoding Index 28804
Index 28804 extracted
Start decoding Index 28806
Index 28806 extracted
Start decoding Index 28807
Index 28807 extracted
Start decoding Index 28809
Index 28809 extracted
Start decoding Index 28810
Index 28810 extracted
Start decoding Index 28812
Index 28812 extracted
Start decoding Index 28813
Index 28813 extracted
Start decoding Index 28815
Index 28815 extracted
Start decoding Index 28816
Index 28816 extracted
Start decoding Index 28818
Index 28818 extracted
Start decoding Index 28819
Index 28819 extracted
Start decoding Index 28821
Index 28821 extracted
Start decoding Index 28822
Index 28822 extracted
Start decoding Index 28824
Index 28824 extracted
Start decoding Index 28825
Index 28825 extracted
Start decoding Index 28827
Index 28827 extracted
Start decoding Index 28828
Index 28828 extracted
Start decoding Index 28830
Index 28830 extracted
Start decoding Index 28831
Index 28831 extracte

Index 29161 extracted
Start decoding Index 29163
Index 29163 extracted
Start decoding Index 29164
Index 29164 extracted
Start decoding Index 29166
Index 29166 extracted
Start decoding Index 29167
Index 29167 extracted
Start decoding Index 29169
Index 29169 extracted
Start decoding Index 29170
Index 29170 extracted
Start decoding Index 29172
Index 29172 extracted
Start decoding Index 29173
Index 29173 extracted
Start decoding Index 29175
Index 29175 extracted
Start decoding Index 29176
Index 29176 extracted
Start decoding Index 29178
Index 29178 extracted
Start decoding Index 29179
Index 29179 extracted
Start decoding Index 29181
Index 29181 extracted
Start decoding Index 29182
Index 29182 extracted
Start decoding Index 29184
Index 29184 extracted
Start decoding Index 29185
Index 29185 extracted
Start decoding Index 29187
Index 29187 extracted
Start decoding Index 29188
Index 29188 extracted
Start decoding Index 29190
Index 29190 extracted
Start decoding Index 29191
Index 29191 extracte

Index 29524 extracted
Start decoding Index 29526
Index 29526 extracted
Start decoding Index 29528
Index 29528 extracted
Start decoding Index 29530
Index 29530 extracted
Start decoding Index 29532
Index 29532 extracted
Start decoding Index 29534
Index 29534 extracted
Start decoding Index 29536
Index 29536 extracted
Start decoding Index 29538
Index 29538 extracted
Start decoding Index 29540
Index 29540 extracted
Start decoding Index 29542
Index 29542 extracted
Start decoding Index 29544
Index 29544 extracted
Start decoding Index 29546
Index 29546 extracted
Start decoding Index 29548
Index 29548 extracted
Start decoding Index 29550
Index 29550 extracted
Start decoding Index 29552
Index 29552 extracted
Start decoding Index 29554
Index 29554 extracted
Start decoding Index 29556
Index 29556 extracted
Start decoding Index 29558
Index 29558 extracted
Start decoding Index 29560
Index 29560 extracted
Start decoding Index 29562
Index 29562 extracted
Start decoding Index 29564
Index 29564 extracte

Index 29810 extracted
Start decoding Index 29812
Index 29812 extracted
Start decoding Index 29813
Index 29813 extracted
Start decoding Index 29815
Index 29815 extracted
Start decoding Index 29816
Index 29816 extracted
Start decoding Index 29818
Index 29818 extracted
Start decoding Index 29819
Index 29819 extracted
Start decoding Index 29821
Index 29821 extracted
Start decoding Index 29822
Index 29822 extracted
Start decoding Index 29824
Index 29824 extracted
Start decoding Index 29825
Index 29825 extracted
Start decoding Index 29827
Index 29827 extracted
Start decoding Index 29828
Index 29828 extracted
Start decoding Index 29830
Index 29830 extracted
Start decoding Index 29831
Index 29831 extracted
Start decoding Index 29833
Index 29833 extracted
Start decoding Index 29834
Index 29834 extracted
Start decoding Index 29836
Index 29836 extracted
Start decoding Index 29837
Index 29837 extracted
Start decoding Index 29839
Index 29839 extracted
Start decoding Index 29840
Index 29840 extracte

Index 30196 extracted
Start decoding Index 30198
Index 30198 extracted
Start decoding Index 30200
Index 30200 extracted
Start decoding Index 30202
Index 30202 extracted
Start decoding Index 30204
Index 30204 extracted
Start decoding Index 30206
Index 30206 extracted
Start decoding Index 30218
Index 30218 extracted
Start decoding Index 30221
Index 30221 extracted
Start decoding Index 30224
Index 30224 extracted
Start decoding Index 30227
Index 30227 extracted
Start decoding Index 30230
Index 30230 extracted
Start decoding Index 30233
Index 30233 extracted
Start decoding Index 30236
Index 30236 extracted
Start decoding Index 30239
Index 30239 extracted
Start decoding Index 30242
Index 30242 extracted
Start decoding Index 30256
Index 30256 extracted
Start decoding Index 30259
Index 30259 extracted
Start decoding Index 30262
Index 30262 extracted
Start decoding Index 30265
Index 30265 extracted
Start decoding Index 30268
Index 30268 extracted
Start decoding Index 30271
Index 30271 extracte

In [18]:
data_et = convertToNumeric(data_et, ['x', 'y', 't', 't_task'])

In [19]:
data_et.groupby(['run_id', 'trial_index'])['x', 'y'].count()

<ipython-input-19-5a412f0f98d6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_et.groupby(['run_id', 'trial_index'])['x', 'y'].count()


x   y
run_id trial_index        
1.0    18.0          6   6
       20.0         11  11
       22.0         11  11
       24.0         11  11
       26.0         10  10
...                 ..  ..
2013.0 501.0        38  38
       503.0        22  22
       504.0        14  14
       506.0        22  22
       507.0        17  17

[14488 rows x 2 columns]

# data_subject

In [20]:
df1 = data_raw.loc[
    :, 
    [
        'run_id',
        'age', 'browliner', 'browser', 'browser_version', 'degree', 'device',
        'ethnic', 'eyeliner', 'eyeshadow', 'gender', 'glasses', 'keptHead',
        'masquara', 'optionalNote', 'platform', 'platform_version', 'sight',
        'triedChin', 'user_agent', 'vertPosition', 'webcam_fps',
        'webcam_height', 'webcam_label', 'webcam_width'
    ]
].drop_duplicates()
    
df2 = data_trial.loc[
    : ,
    [
        'chinFirst', 'choiceTask_amountLeftFirst', 'chosenAmount',
        'chosenDelay', 'run_id', 'prolificID'
    ]
].drop_duplicates()
    
data_subject = df1.merge(df2, on='run_id', how='left') \
    .rename(columns={'age': 'birthyear'})

## Add prolific data

In [21]:
data_prolific = pd.read_csv(r'C:/Users/User/Google Drive/Masterarbeit/Acquise/prolific_documents/prolific_export.csv') \
    .rename(columns={'participant_id': 'prolificID'})

data_subject = data_subject.merge(data_prolific, on='prolificID', how='left')

In [22]:
data_subject['status'] = data_subject['status'].fillna('NOTPROLIFIC')

In [23]:
data_subject.loc[:, ['run_id', 'prolificID']]

,run_id,prolificID
0,1,5fccc8ac636416a4288a9f3d
1,103,600063f2943eab0acc812ed8
2,106,NaN
3,108,NaN
4,11,5c5684ef9d244c0001b29f1e
...,...,...
79,2012,fdolshwtfs
80,2013,amsdkerlxp
81,2002,Tim2
82,2008,Studie1970


# Export data

In [24]:
if not os.path.exists('./data_jupyter'):
    os.mkdir('./data_jupyter')

data_et.to_csv("data_jupyter/data_et.csv", index=False, header=True)
data_trial.to_csv("data_jupyter/data_trial.csv", index=False, header=True)
data_subject.to_csv("data_jupyter/data_subject.csv", index=False, header=True)

# Feedback

In [25]:
print('Success! Script ran through')

Success! Script ran through
